In [91]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# Name - Rev.get_R_est
#__author__ = 'Prakhar MISRA'
# Created 8/18/2019
# Last edit 8/18/2010

# code to answer reviwer commment regarding the inability of the model toaccount for secondary PM2.5 
# hence following Upadhaye and dey 2018, we will also include meteorology 

# Function to multiply the cities with corresponding beta to find the Rest
# also known as get beta


# Important resource

# following course by PSU teaches statistics with derived examples
# Confidence interval -  https://newonlinecourses.science.psu.edu/stat414/node/297/
# prediction interval - https://newonlinecourses.science.psu.edu/stat414/node/298/
# CI for Multi linear regression https://newonlinecourses.science.psu.edu/stat501/lesson/7/7.1

# for bayesian 
# More advanced , posterioir predictive distribution for Bayesian 
# https://baezortega.github.io/2018/08/06/robust_regression/

import pandas as pd
import os
import scipy as sp
import scipy.stats as stats
import matplotlib.pyplot as plt
import numpy as np
from datetime import timedelta, date
import matplotlib.dates as mdates
import time
import netCDF4
import datetime as dt
from netCDF4 import MFDataset
from netCDF4 import num2date
import datetime as dt  # Python standard library datetime  module
from glob import glob

currdir = os.getcwd()
import seaborn as sns
sns.set_style("whitegrid")



%matplotlib notebook

gpath = r"D:\Research\Codes_W\GoogleTrends\\"
currdir = os.getcwd()



In [72]:

# dtaa path 
reanalysis_dir = os.path.join("E:/", "OneDrive", "AQM_Research", "Data", "Raw", "NCEP_globalreanalysis")
reanalysis_dir = os.path.join("G:/", "AQM_Research", "Data", "Data_raw", "Weather", "NCEP_globalreanalysis")


# list of meteorological characteristics
list_meteor = ["tmax", "tmin", "rhum", "uwnd", "vwnd", "hpbl"]

#meteor path for monthly data
tmaxfile = os.path.join(reanalysis_dir, "tmax.2m.mon.mean.nc") # + "2011.nc"
tminfile = os.path.join(reanalysis_dir, "tmin.2m.mon.mean.nc")
rhumfile = os.path.join(reanalysis_dir, "rhum.mon.mean.nc")
uwndfile = os.path.join(reanalysis_dir, "uwnd.10m.mon.mean.nc")
vwndfile = os.path.join(reanalysis_dir, "vwnd.10m.mon.mean.nc")

# NOAA-CIRES 20th Century Reanalysis version 2c Monthly Averages
# https://www.esrl.noaa.gov/psd/cgi-bin/db_search/DBSearch.pl?Variable=Planetary+Boundary+Layer+Height&group=0&submit=Search
pblfile = os.path.join(reanalysis_dir, "hpbl.mon.mean.nc") 

#meteor dict
dict_meteorpath = {
    "tmax":tmaxfile,
    "tmin":tminfile,
    "rhum":rhumfile,
    "uwnd":uwndfile,
    "vwnd":vwndfile,
    "hpbl":pblfile,
}

# file containg lat/lon
df_loc = pd.read_csv(os.path.join(currdir, "lib", "20citycoordforAQmodel.csv"))

In [82]:

DONT RUN AGAIN


#citylist = ["Delhi", "Bangkok", "Kolkata","Tokyo", "Dhaka", "Karachi", "Taipei", "Tehran", "Seoul", "Manila", "HoChiMinh", "Yangon"]
citylist = df_loc.city_coord.unique()


for city in citylist:
    
    # city"s lat, lon
    citylat  =  df_loc.loc[df_loc["city_coord"]==city]["lat"]
    citylon  =  df_loc.loc[df_loc["city_coord"]==city]["lon"]


    for meteor in list_meteor[:-1]:

        meteorseries = []
        timeseries = []


        f = MFDataset(dict_meteorpath[meteor])
        #print (f.variables)

        meteorval = f.variables[meteor][:]
        lats = f.variables['lat'][:]
        lons = f.variables['lon'][:]
        
        # extract and convert time to datetime units
        time = f.variables['time']
        timeseries = num2date(time[:], time.units) 

        # Find the nearest latitude and longitude for the city
        lat_idx = np.abs(lats - citylat.values).argmin()
        lon_idx = np.abs(lons - citylon.values).argmin()
        
     

        #get the values for the year
        meteorseries = meteorval[:, :, lat_idx, lon_idx]

        #print (meteorval.shape) # (2922, 73, 144) equivalent to 8 year data
        #print (time.units) # hours since 1800-01-01 00:00:0.0

        asd = pd.DataFrame({"date": timeseries, meteor: meteorseries[:,0]} )
        asd["city_coord"] = city
        asd.set_index("date", inplace= True)

        asd.to_csv(os.path.join(currdir, "Data",  "Reanalysis","Monthly" , city + "_" + meteor + ".csv"))
        
        

In [83]:

DONT RUN AGAIN

# another specially for PBL 

for city in citylist:
    
    # city"s lat, lon
    citylat  =  df_loc.loc[df_loc["city_coord"]==city]["lat"]
    citylon  =  df_loc.loc[df_loc["city_coord"]==city]["lon"]


    meteor = "hpbl"

    meteorseries = []
    timeseries = []


    f = MFDataset(dict_meteorpath[meteor])
    #print (f.variables)

    meteorval = f.variables[meteor][:]
    lats = f.variables['lat'][:]
    lons = f.variables['lon'][:]

    # extract and convert time to datetime units
    time = f.variables['time']
    timeseries = num2date(time[:], time.units) 

    # Find the nearest latitude and longitude for the city
    lat_idx = np.abs(lats - citylat.values).argmin()
    lon_idx = np.abs(lons - citylon.values).argmin()



    #get the values for the year
    meteorseries = meteorval[ :, lat_idx, lon_idx]

    #print (meteorval.shape) # (2922, 73, 144) equivalent to 8 year data
    #print (time.units) # hours since 1800-01-01 00:00:0.0

    asd = pd.DataFrame({"date": timeseries, meteor: meteorseries} )
    asd["city_coord"] = city

    asd.set_index("date", inplace= True)

    asd.to_csv(os.path.join(currdir, "Data",  "Reanalysis","Monthly" , city + "_" + meteor + ".csv"))

        

In [163]:
# combine all od them together


list_df_city_met =[]
for meteor in list_meteor:
    
    list_df = []
    for file in glob(os.path.join(currdir, "Data",  "Reanalysis","Monthly", '*'+ meteor + '.csv')):

        df_meteor = pd.read_csv(file, header = 0)
        df_meteor["date"] = pd.to_datetime(df_meteor['date'], format="%Y-%m-%d")
        df_meteor = df_meteor[df_meteor.date.dt.year>=1980]
        df_meteor.index = pd.MultiIndex.from_arrays(df_meteor[['city_coord', 'date']].values.T, names=['idxR1', 'idxR2'])
        #df_meteor = df_meteor.set_index("date")

        list_df.append(df_meteor)
    
    temp_df = pd.concat(list_df)
    #temp_df.index = pd.MultiIndex.from_arrays(temp_df[['city', 'date']].values.T, names=['idxR1', 'idxR2'])
    list_df_city_met.append(temp_df)

# merge all the dfferent meteors in axis = 1 this axis (horizontal) -->
df_city_met = pd.concat(list_df_city_met, axis =1)

# remove duplicate columns
df = df_city_met
df = df.loc[:,~df.columns.duplicated()]

#correct the temp by Kelvin
df["tmax"] = df["tmax"] - 273
df["tmin"] = df["tmin"] - 273
df["wnd"] = (df["uwnd"]**2 + df["vwnd"]**2)**0.5

# since PBL values are missing after 2014, replace successive years by vaues of 2014
df.loc[df.date.dt.year== 2015, ["hpbl"]] = df[df.date.dt.year== 2014]["hpbl"].values
df.loc[df.date.dt.year== 2016, ["hpbl"]] = df[df.date.dt.year== 2014]["hpbl"].values
df.loc[df.date.dt.year== 2017, ["hpbl"]] = df[df.date.dt.year== 2014]["hpbl"].values
# and save
df.to_csv(os.path.join(currdir, "Data",  "Reanalysis","Monthly", '0_allcity_meteor.csv'))

C:\Users\Prakhar\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Prakhar\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Prakhar\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#

In [177]:
# join the meteordata with the file to be used for the bayesian simulation 


# open the city meteor data file
df_city_meteor = pd.read_csv(os.path.join(currdir, "Data",  "Reanalysis","Monthly", '0_allcity_meteor.csv'), header = 0)
df_city_meteor["date"] = pd.to_datetime(df_city_meteor['date'], format="%Y-%m-%d")
# set the multi index
df_city_meteor.index = pd.MultiIndex.from_arrays(df_city_meteor[['city_coord', 'date']].values.T, names=['idx1', 'idx2'])


# open the city data for Bayesian 
# df_allAYsetclean = pd.read_csv(os.path.join(currdir, "my_bayesian",  "HBM_city_tier_20180116","report20180212_IDW2_20190814", 'allAY20180212IDW2setclean20012011.csv'))
df_allAYsetclean = pd.read_csv(os.path.join(currdir, "my_bayesian",  "HBM_city_tier_20180116","report20180212_IDW2_20190814", "meteorology_model", 'allAY20180212IDW2setclean.csv'))
df_allAYsetclean["date"] = pd.to_datetime(df_allAYsetclean['date'], format="%m/%d/%Y")
# set the mutu index
df_allAYsetclean.index = pd.MultiIndex.from_arrays(df_allAYsetclean[['citychk_loc', 'date']].values.T, names=['idxR1', 'idxR2'])
df_allAYsetclean.drop(columns = ["citychk_loc",  "date"], inplace = True)

# and merge on the basis of multiindex

# need to give name to index level
df_allAYsetclean.index.levels[0].name = "idx1"
df_allAYsetclean.index.levels[1].name = "idx2"
df_city_meteor.index.levels[0].name = "idx1"
df_city_meteor.index.levels[1].name = "idx2"


# now merge
df_merge = pd.merge(df_allAYsetclean, df_city_meteor, how='inner', on=None, left_on=None, right_on=None,
         left_index=True, right_index=True, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

# and save
# df_merge.to_csv(os.path.join(currdir, "my_bayesian",  "HBM_city_tier_20180116","report20180212_IDW2_20190814", 'allAY20180212IDW2setclean20012011_meteor.csv'))
df_merge.to_csv(os.path.join(currdir, "my_bayesian",  "HBM_city_tier_20180116","report20180212_IDW2_20190814", "meteorology_model", 'allAY20180212IDW2setclean_meteor.csv'))